In [1]:
pip install requests pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gspread oauth2client


Note: you may need to restart the kernel to use updated packages.


In [14]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import pytz  # This module provides timezone definitions

# Use the same scope as before
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Use your downloaded json file with your Google API credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\gkers\Documents\Jupyter\intechgui-68cb3ebdb2d2.json", scope)
client = gspread.authorize(creds)

# Access the Google Sheet using the URL
sheet_url = "https://docs.google.com/spreadsheets/d/10hzD9efmuT1bAeipZc3cwfAl8s7nlxu2ZbFbX1bWMQU/edit#gid=0"
sheet = client.open_by_url(sheet_url)

# Choose the right sheet or worksheet if there are multiple ones
worksheet = sheet.get_worksheet(0)  # `0` refers to the first sheet within the spreadsheet

# Get all the values from the sheet into a DataFrame
data = worksheet.get_all_values()
headers = data.pop(0)  # Assumes first row is header
df = pd.DataFrame(data, columns=headers)

# Convert columns to appropriate data types
df['DateTime'] = pd.to_datetime(df['Time'], format='%y%m%d%H%M%S', errors='coerce')  # Convert 'Time' to datetime
df['Battery Voltage'] = pd.to_numeric(df['Battery Voltage'], errors='coerce')  # Convert 'Battery Voltage' to numeric
df['Water Height'] = pd.to_numeric(df['Water Height'], errors='coerce')  # Convert 'Water Height' to numeric
df['Panel Power'] = pd.to_numeric(df['Panel Power'], errors='coerce')  # Convert 'Panel Power' to numeric

# Make the DateTime column timezone aware (assuming the time is in UTC)
df['DateTime'] = df['DateTime'].dt.tz_localize('UTC')

# Convert from UTC to Mountain Time
df['DateTime'] = df['DateTime'].dt.tz_convert('America/Denver')

# Optionally, print the DataFrame to check the correct data types and values
print(df.dtypes)  # Print data types to confirm
print(df.head())  # Print first few rows of the DataFrame


Node Identifier                            object
Time                                       object
Water Height                              float64
Battery Voltage                           float64
Panel Power                                 int64
Processor Temp                             object
Sleep Mode                                 object
DateTime           datetime64[ns, America/Denver]
dtype: object
  Node Identifier          Time  Water Height  Battery Voltage  Panel Power  \
0            N001  240319080000           NaN            12.75            0   
1            N001  240319090000           NaN            12.74            0   
2            N001  240319100000           NaN            12.71            0   
3            N001  240319110000           NaN            12.70            0   
4            N001  240319120000           NaN            12.67            0   

  Processor Temp Sleep Mode                  DateTime  
0              0          0 2024-03-19 02:00:00-06:00  

In [19]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Set up your Dash application
app = dash.Dash(__name__)

# Define Node Coordinates (you might want to update these based on actual data or another column in df)
node_coords = {
    'N001': {'lat': 40.011316, 'lon': -105.243969},
    'N002': {'lat': 40.085177, 'lon': -105.219921},
    'N003 (Fake Data)': {'lat': 39.962842, 'lon': -105.503144}
}

# Graph styling function
def style_graph(fig):
    fig.update_layout(
        plot_bgcolor='#2b2b2b',
        paper_bgcolor='#2b2b2b',
        font=dict(color='#FFF', family='Roboto Mono'),
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(title_text='', color='lightgray', showgrid=False, gridcolor='#505050'),
        yaxis=dict(title_text='Value', title_font=dict(size=16), color='lightgray', showgrid=True, gridcolor='#505050'),
        title=dict(font=dict(size=20)),
    )
    fig.update_traces(line=dict(color='#FFD700'))  # Yellow line color
    fig.update_layout(hoverlabel=dict(bgcolor='black', font_size=12, font_family='Roboto Mono', font_color='lightgray'))
    return fig


app.layout = html.Div(style={
    'backgroundColor': '#2b2b2b', 
    'color': '#DDD', 
    'fontFamily': 'Roboto Mono', 
    'display': 'flex',
    'flexDirection': 'row',
    'alignItems': 'flex-start'
}, children=[
    # Left Column
    html.Div(style={
        'flex': '1', 
        'padding': '20px',
        'display': 'flex',
        'flexDirection': 'column'
    }, children=[
        html.Img(
            src="/assets/INTECH_HORIZONTAL.png",
            style={'height': 'auto', 'width': '100%', 'maxWidth': '600px', 'display': 'block', 'marginBottom': '15px'}
        ),
        html.H1("INTECH Interferometric Reflectometry Dashboard", style={'color': 'white'}),
        html.Div(style={'margin': '15px 0'}, children=[  # This div adds margin around the dropdown
            html.Label("Select Node:", style={'color': '#FFF', 'fontSize': 20}),
            dcc.Dropdown(
                id='node-dropdown',
                options=[{'label': node, 'value': node} for node in df['Node Identifier'].unique()],
                value='N001',
                style={
                    'backgroundColor': '#333', 
                    'color': '#FFF',
                    'fontSize': 20,
                },
                clearable=True,
                placeholder="Select a node...",
                className='dropdown'
            ),
        ]),
        dcc.Graph(id='map', style={'backgroundColor': '#333', 'marginBottom': '15px'}),
        html.H1("Send Quality Control Variables to the Node", style={'color': 'white', 'marginBottom': '10px'}),
        html.Label("Elevation Angle Mask Range:", style={'color': '#FFF'}),
        dcc.RangeSlider(
            id='emask-range',
            min=0,
            max=30,
            step=2.5,
            value=[5, 15],
            className='slider-style'  # Use this class to style the slider in CSS
        ),
        html.Label("Azimuth Angle Mask Range:", style={'color': '#FFF'}),
        dcc.RangeSlider(
            id='amask-range',
            min=0,
            max=360,
            step=30,
            value=[90, 270],
            className='slider-style'
        ),
        html.Label("e_diff:", style={'color': '#FFF'}),
        dcc.Slider(
            id='e-diff-slider',
            min=3,
            max=25,
            step=1,
            value=5,
            className='slider-style'
        ),
        html.Button(
        'Send Configuration', 
        id='save-button', 
        n_clicks=0,
        style={
            'fontSize': '18px',
            'fontFamily': 'Roboto Mono',
            'padding': '10px 20px', 
            'color': '#333',
            'backgroundColor': '#FFD700',
            'border': 'none',
            'cursor': 'pointer',
            'marginTop': '15px'
        }
    )
    ]),

    # Right Column
    html.Div(style={
        'flex': '2',  # Adjust the ratio as needed
        'display': 'grid',
        'gridTemplateRows': 'auto auto auto',  # Three rows
        'gridGap': '20px',  # Space between rows
        'padding': '20px',
    }, children=[
        dcc.Graph(id='graph-water-height', style={'gridRow': '1'}),
        dcc.Graph(id='graph-battery-voltage', style={'gridRow': '2'}),
        dcc.Graph(id='graph-panel-power', style={'gridRow': '3'}),
    ])
])

@app.callback(
    Output('map', 'figure'),
    [Input('node-dropdown', 'value')]
)
def update_map(selected_node):
    lat = node_coords[selected_node]['lat']
    lon = node_coords[selected_node]['lon']
    
    # Create a DataFrame for the selected node
    node_df = pd.DataFrame([{'lat': lat, 'lon': lon, 'hover_name': selected_node}])

    # Create the figure using Plotly Express
    fig = px.scatter_mapbox(
        node_df,
        lat='lat',
        lon='lon',
        hover_name='hover_name',
        zoom=15,
        height=700,
        mapbox_style="satellite", # Use the dark mode style
    )
    
    # Update layout to have no margins and match the dark theme
    fig.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        mapbox_accesstoken='pk.eyJ1IjoiZ2tlcnNleTEwNiIsImEiOiJjbHV1OGY2cXcwN25wMmxvOTAxbXB3azYxIn0.ppr7W_ticC4BF2iHyN7VYw'
    )
    # Update the marker size and color to match the yellow color used in your graphs
    fig.update_traces(marker=dict(size=15, color='#FFD700'))
    return fig

@app.callback(
    [Output('graph-water-height', 'figure'),
     Output('graph-battery-voltage', 'figure'),
     Output('graph-panel-power', 'figure')],
    [Input('node-dropdown', 'value')]
)
def update_graphs(selected_node):
    filtered_df = df[df['Node Identifier'] == selected_node]
    # Create the figures
    fig_water_height = px.line(filtered_df, x='DateTime', y='Water Height', title='Water Level Over Time')
    fig_battery_voltage = px.line(filtered_df, x='DateTime', y='Battery Voltage', title='Battery Voltage Over Time')
    fig_panel_power = px.line(filtered_df, x='DateTime', y='Panel Power', title='Panel Power Over Time')

    # Apply the styling to match the theme
    fig_water_height = style_graph(fig_water_height)
    fig_battery_voltage = style_graph(fig_battery_voltage)
    fig_panel_power = style_graph(fig_panel_power)

    return [fig_water_height, fig_battery_voltage, fig_panel_power]

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)